In [8]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
!pip install emoji==1.5.0

  Using cached emoji-1.5.0-py3-none-any.whl
  Attempting uninstall: emoji
    Found existing installation: emoji 1.7.0
    Uninstalling emoji-1.7.0:
      Successfully uninstalled emoji-1.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
emoji-translate 0.1.1 requires emoji==1.7.0, but you have emoji 1.5.0 which is incompatible.


In [10]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install emoji_translate

  Using cached emoji-1.7.0-py3-none-any.whl
  Attempting uninstall: emoji
    Found existing installation: emoji 1.5.0
    Uninstalling emoji-1.5.0:
      Successfully uninstalled emoji-1.5.0
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pytorch-lightning -q -U

Note: you may need to restart the kernel to use updated packages.


In [92]:
import pandas as pd
import re
import emoji
import numpy as np
from emoji_translate.emoji_translate import Translator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch.nn as nn
import torch
import torch.optim as optim
import pytorch_lightning as pl
emo = Translator(exact_match_only=False, randomize=True)

# Intersection code, the DataFrame should be empty

In [93]:
df_test = pd.read_csv("/kaggle/input/testdatadeslab/hindi_test.csv")
df_train = pd.read_csv("/kaggle/input/traindatadeslab/hindi_train_val.csv")

df_test = df_test.merge(df_train, on="text")
df_test# Intersection code, the DataFrame should be empty

,label_x,text,label_y


In [94]:
def pre_processing(line):
    tokens = line.split()
    processed_text = " ".join(tokens)
    processed_text = "".join((' '+c+' ') if c in emoji.UNICODE_EMOJI['en'] else c for c in processed_text)
    processed_text = emo.demojify(processed_text)
    return processed_text

In [95]:
model_name = 'bert-base-multilingual-cased'


In [96]:
df = pd.read_csv("/kaggle/input/lstmsample/train.csv")
df['text'] = df['text'].apply(pre_processing)
labels = df["label"].values.tolist()
text = df['text'].values.tolist()

In [97]:
train_text,val_text,train_label,val_label = train_test_split(text,labels,random_state=5,test_size=.2)
# val_text,test_text,val_label,test_label = train_test_split(val_text,val_label,random_state=5,test_size=.2)

In [98]:
# device (turn on GPU acceleration for faster execution)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device used: {}.".format(device))

Device used: cuda.


In [99]:
from torch import nn
from transformers import AutoModel

in_features = 768 # it's 768 because that's the size of the output provided by the underlying BERT model

class BertWithCustomNNClassifier(nn.Module):
    """
    A pre-trained BERT model with a custom classifier.
    The classifier is a neural network implemented in this class.
    """
    
    def __init__(self, linear_size):
        super(BertWithCustomNNClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout1 = nn.Dropout(p=0.5)
        self.linear1 = nn.Linear(in_features=in_features, out_features=linear_size)
        self.batch_norm1 = nn.BatchNorm1d(num_features=linear_size)
        self.dropout2 = nn.Dropout(p=0.8)
        self.linear2 = nn.Linear(in_features=linear_size, out_features=1)
        self.batch_norm2 = nn.BatchNorm1d(num_features=1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, tokens, attention_mask):
        bert_output = self.bert(input_ids=tokens, attention_mask=attention_mask)
        x = self.dropout1(bert_output[1])
        x = self.linear1(x)
        x = self.dropout2(x)
        x = self.batch_norm1(x)
        x = self.linear2(x)
        x = self.batch_norm2(x)
        return self.sigmoid(x)
        
    def freeze_bert(self):
        """
        Freezes the parameters of BERT so when BertWithCustomNNClassifier is trained
        only the wieghts of the custom classifier are modified.
        """
        for param in self.bert.named_parameters():
            param[1].requires_grad=False
    
    def unfreeze_bert(self):
        """
        Unfreezes the parameters of BERT so when BertWithCustomNNClassifier is trained
        both the wieghts of the custom classifier and of the underlying BERT are modified.
        """
        for param in self.bert.named_parameters():
            param[1].requires_grad=True

            
print(BertWithCustomNNClassifier.__doc__)


class BertClassifier(nn.Module):
  def __init__(self, dropout=0.5):
    super(BertClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(768,2)
    self.sigmod = nn.Sigmoid()
  
  def forward(self, input_id, mask):
    _, pooled_output = self.bert(input_ids = input_id, attention_mask = mask, return_dict = False)
    dropout_output = self.dropout(pooled_output)
    linear_output = self.linear(dropout_output)
    final_layer = self.sigmod(linear_output)
    return final_layer


    A pre-trained BERT model with a custom classifier.
    The classifier is a neural network implemented in this class.
    


In [100]:
# parameters
num_of_epochs = 10
learning_rate = 1e-3
batch_size = 16
hidden_layers = 8

print("Epochs: {}".format(num_of_epochs))
print("Learning rate: {:.6f}".format(learning_rate))
print("Batch size: {}".format(batch_size))
print("The number of hidden layers in the custom head: {}".format(hidden_layers))

Epochs: 10
Learning rate: 0.001000
Batch size: 16
The number of hidden layers in the custom head: 8


In [101]:
model = BertWithCustomNNClassifier(linear_size=hidden_layers)
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertWithCustomNNClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [102]:
# from transformers import AdamW

# # optimizer
# optimizer = AdamW(model.parameters(), lr=learning_rate)
# print('Initialized optimizer.')

In [103]:
# loss function
# loss_fn = nn.BCELoss()
# print('Initialized loss function.')

In [104]:
# from transformers import AutoTokenizer


# tokenizer = AutoTokenizer.from_pretrained(model_name)


# train_encoding = tokenizer(train_text, truncation=True, padding=True,max_length=512,return_tensors="pt")
# val_encoding = tokenizer(val_text, truncation=True, padding=True,max_length=512,return_tensors="pt")
# test_encoding = tokenizer(test_text, truncation=True, padding=True,max_length=512,return_tensors="pt")

In [105]:
# print(type(train_encodings[0]["input_ids"]))
# train_encoding = train_encoding[0]
#  val_encodings = val_encodings[0]

In [122]:
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
class SentimentDataset(torch.utils.data.Dataset):
    """
    Class to store the Sentece data as PyTorch Dataset
    """
    
    def __init__(self,df):
        self.labels = df['label']
        self.texts = [tokenizer(
        text,
        max_length = 256,
        truncation = True,
        return_tensors = "pt",
        padding = 'max_length'
        ) for text in df['text']]

    def __getitem__(self,idx):
        batch_texts = self.get_batch_texts(idx)
        batch_labels = self.get_batch_labels(idx)
        return batch_texts, batch_labels
    def __len__(self):
        return len(self.labels)
    def classes(self):
        return self.labels

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def get_batch_labels(self,idx):
        return np.array(self.labels[idx])    

print(SentimentDataset.__doc__)


    Class to store the Sentece data as PyTorch Dataset
    


# Hindi-BERT

In [123]:
# from torch.utils.data import DataLoader

# # Dataset & dataloader
# train_dataset = SentimentDataset(train_encoding, train_label)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataset = SentimentDataset(val_encoding, val_label)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# print('Created train & val datasets.')
# # test_dataset = SentimentDataset(test_encoding, val_label)
# # test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [124]:
def training_step(dataloader, model, optimizer, loss_fn, if_freeze_bert):
    """Method to train the model"""
    
    model.train()
    model.freeze_bert() if if_freeze_bert else model.unfreeze_bert()
      
    epoch_loss = 0
    size = len(dataloader.dataset)
 
    for i, batch in enumerate(dataloader):        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
    
        outputs = torch.flatten(model(tokens=input_ids, attention_mask=attention_mask))
                        
        optimizer.zero_grad()
        loss = loss_fn(outputs, labels.float())
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

            
print(training_step.__doc__)

Method to train the model


In [125]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def eval_prediction(y_batch_actual, y_batch_predicted):
    """Return batches of accuracy and f1 scores."""
    y_batch_actual_np = y_batch_actual.cpu().detach().numpy()
    y_batch_predicted_np = np.round(y_batch_predicted.cpu().detach().numpy())
    
    acc = accuracy_score(y_true=y_batch_actual_np, y_pred=y_batch_predicted_np)
    f1 = f1_score(y_true=y_batch_actual_np, y_pred=y_batch_predicted_np, average='weighted')
    
    return acc, f1

# print(eval_prediction.__doc__)
def validation_step(dataloader, model, loss_fn):
    """Method to test the model's accuracy and loss on the validation set"""
    
    model.eval()
    model.freeze_bert()
    
    size = len(dataloader)
    f1, acc = 0, 0
    
    with torch.no_grad():
        for batch in dataloader:
            X = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            y = batch['labels'].to(device)
                  
            pred = model(tokens=X, attention_mask=attention_mask)
            
            acc_batch, f1_batch = eval_prediction(y.float(), pred)                        
            acc += acc_batch
            f1 += f1_batch

        acc = acc/size
        f1 = f1/size
                
    return acc, f1
        
print(validation_step.__doc__)

Method to test the model's accuracy and loss on the validation set


In [126]:
# from tqdm.auto import tqdm

# tqdm.pandas()

# best_acc, best_f1 = 0, 0
# path = './best_model.pt'
# if_freeze_bert = False

# for i in tqdm(range(num_of_epochs)):
#     print("Epoch: #{}".format(i+1))

# #     if i < 5:
# #         if_freeze_bert = False
# #         print("Bert is not freezed")
# #     else:
# #         if_freeze_bert = True
# #         print("Bert is freezed")
    
#     training_step(train_loader, model,optimizer, loss_fn, if_freeze_bert)
#     train_acc, train_f1 = validation_step(train_loader, model, loss_fn)
#     val_acc, val_f1 = validation_step(val_loader, model, loss_fn)
    
#     print("Training results: ")
#     print("Acc: {:.3f}, f1: {:.3f}".format(train_acc, train_f1))
    
#     print("Validation results: ")
#     print("Acc: {:.3f}, f1: {:.3f}".format(val_acc, val_f1))
    
#     if val_acc > best_acc:
#         best_acc = val_acc    
        
#         checkpoint = {'model': model,
#           'state_dict': model.state_dict()}

#         torch.save(checkpoint, 'checkpoint.pth')

# Testing

In [127]:
from sklearn.metrics import accuracy_score, f1_score
def evaluate(model, test_data):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    test = SentimentDataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=64)



    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    y_pred = []
    y_true = []
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
              optim_labels = test_label
              optim_ids = test_input['input_ids']
              mask = test_input['attention_mask'].to(device)
              test_label = test_label.to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              y_pred.extend(output.argmax(dim=1).tolist())
              y_true.extend(test_label.tolist())
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
        
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print(f'Test F1 Score: {f1_score(y_true, y_pred): .3f}')

#df_test.reset_index(drop = True,inplace = True)
#evaluate(model, df_test)

In [128]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

model = load_checkpoint('/kaggle/input/lstmsamplemodel/checkpoint.pth')

In [129]:
df = pd.read_csv("/kaggle/input/testdatadeslab/hindi_test.csv")
# df = pd.read_csv('/kaggle/input/dlabdata/hindi_train_val.csv')
X_eval = df.drop(['label'], axis = 1)
y_eval = df['label']
df_eval = pd.concat([X_eval, y_eval], axis=1)
df_eval.reset_index(drop = True,inplace = True)
evaluate(model, df_eval)

Test Accuracy:  0.817
Test F1 Score:  0.802
